# Lista 2 - Regressão Logística e Classificadores Estatísticos

## Sumário

- Questão 1: [A](#Questão-1-a.), [B](#Questão-1-b.)
- Questão 2: [A](#Questão-2-a.), [B](#Questão-2-b.)

<span style="position: absolute; top: 10px; right: 10px; background: green; padding: 0.5em; color: white; border-radius: 8px; font-weight: bold">Vaux Gomes</span>

## Implementações

### Importações

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sklearn.preprocessing
from sklearn.base import \
    BaseEstimator, RegressorMixin
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

### Experimento

In [2]:
def split(data, perc=0.66):
    ''' Splits the data into two peaces '''
    idx = int(perc * data.shape[0])
    return data[:idx], data[idx:]

In [3]:
def cross_val(clf, X, y, folds=10, stats=False, progress=False):
    ''' Runs a full n-fold cross-validation on clf '''
    
    # Auxiliary
    accuracy = {'global': [], 'class': []}
    p_size = X.shape[0]/folds # partition size (float)
    
    # Normalizer
    normalizer = MinMax()

    # Cross validation
    for n in range(folds):
        # Partition boundaries
        start, end = int(n * p_size), int((n + 1) * p_size)

        # Normalized Train
        X_train = normalizer.fit_transform(np.concatenate([X[:start], X[end:]]))
        y_train = np.concatenate([y[:start], y[end:]])

        # Normalized Validation
        X_valid = normalizer.fit_transform(X[start:end])
        y_valid = y[start:end]

        # Training
        clf.fit(X_train, y_train)

        # Prediction
        y_hat = clf.predict(X_valid)

        # Confusion Matrix
        c_matrix = confusion_matrix(y_valid, y_hat)

        # Accuracy
        acc = c_matrix.diagonal().sum()/c_matrix.sum()       # General
        accuracy['global'].append(acc)

        acc_class = c_matrix.diagonal()/c_matrix.sum(axis=1) # By class
        accuracy['class'].append(acc_class)
        
        # Progress printing for watching the magic happening
        if progress:
            print(end='-')   
        
    # Printing
    if stats:
        print('-'*(50 - (n if progress else 0)))
        print(f' {clf}')
        print('-'*50)
        print(' Global Accuracy')
        print(f'  - Mean:               {np.mean(accuracy["global"])}')
        print(f'  - Standard deviation: {np.std(accuracy["global"])}')
        print(' Accuracy by class')
        print(f'  - Mean:               {np.mean(accuracy["class"], axis=0)}')
        print(f'  - Standard deviation: {np.std(accuracy["class"], axis=0)}')
        
    # Return
    return accuracy

### MinMax Normalizer

In [4]:
class MinMax():
    ''' MinMax Normalizer '''

    #
    def fit_transform(self, X):
        self.min = X.min(axis=0)
        self.max = X.max(axis=0)
        
        return (X - self.min)/(self.max - self.min)
    
    #
    def transform(self, X):
        return (X - self.min)/(self.max - self.min)
    
    #
    def restore(self, X):
        return X*(self.max - self.min) + self.min

### Binary Logistic Regression via Gradient Descent

In [5]:
class BinaryLogisticRegressionGD(BaseEstimator, RegressorMixin):
    ''' Binary Logistic Regression via Gradient Descent (GD) '''

    #
    def __init__(self, alpha=10**-3, epocs=300):
        self.epocs = epocs
        self.alpha = alpha

    # 
    def fit(self, X, y):
        # Parameters
        self.bias = 0                       # Bias
        self.weights = np.zeros(X.shape[1]) # Weights
        
        # Main loop
        for i in range(self.epocs):
            # Prediction
            y_hat = self.activation(np.matmul(self.weights, X.T) + self.bias)
            
            # Error
            err = y_hat - y
            
            # Gradients
            g_bias = err.mean()
            g_weights = X.T.dot(err).mean(axis=1)
            
            # Update: using 'minus' due error calculation (ŷ - y)
            self.bias -= self.alpha * g_bias
            self.weights -= self.alpha * g_weights
         
    #
    def predict(self, X, R_cost=1):
        ''' R_cost: Decision Limiar '''
        
        y_hat = np.matmul(X, self.weights.T) + self.bias  
        return np.where(self.activation(y_hat) > (1/(R_cost+1)), 1, 0)

    #
    @staticmethod
    def activation(z):
        ''' Sigmoid function '''
        return 1/(1 + np.exp(-z))

### Softmax Regression via Gradient Descent - Multivariate

> Calculei os pesos por classe, mas entendo que tem uma maneira de fazer o cálculo usando o numpy. Só não tive tempo. : (

In [6]:
class SoftmaxRegressionGD(BaseEstimator, RegressorMixin):
    ''' Softmax Regression via Gradient Descent - Multivariate '''

    #
    def __init__(self, alpha=10**-3, epocs=300):
        self.epocs = epocs
        self.alpha = alpha

    #
    def fit(self, X, y):
        # Aux
        self.classes, self.counts = np.unique(y, return_counts=True)
        
        # Parameters
        self.bias = np.zeros(self.classes.shape[0])                  # Bias
        self.weights = np.zeros((self.classes.shape[0], X.shape[1])) # Weights
        
        # Binarization of y
        binarizer = sklearn.preprocessing.LabelBinarizer()
        binarizer.fit(range(self.classes.size))
        #
        y_ = binarizer.transform(y)
        
        # Main loop
        for i in range(self.epocs):
            #
            parts = self.activation(X)
            
            #
            for k, c in enumerate(self.classes):
                # Prediction
                y_hat = parts[k]/parts.sum(axis=0)

                # Error of class k
                err_k = y_hat - y_[:, k:k+1]

                # Gradients
                g_bias = err_k.mean()
                g_weights = X.T.dot(err_k).mean(axis=1)
                
                # Update: using 'minus' due error calculation (ŷ - y)
                self.bias[k] -= self.alpha * g_bias
                self.weights[k] -= self.alpha * g_weights
                
    #
    def predict(self, X):
        #
        y_hat = np.matmul(X, self.weights.T) + self.bias  
        return np.argmax(y_hat, axis=1)

    #
    def activation(self, X):
        ''' Softmax '''
        #
        parts = []
        for i, c in enumerate(self.classes):
            parts.append(np.exp(self.weights[i].T * X + self.bias[i]))

        #
        return np.array(parts)

### Análise do discriminante Gaussiano

In [7]:
class GaussianDiscriminantAnalysis(BaseEstimator, RegressorMixin):
    ''' Gaussian Discriminant Analysis - Multivariate '''
    
    def fit(self, X, y):
        # Classes and Counts
        self.classes, self.counts = np.unique(y, return_counts=True)
        
        # Distributions (Prior)
        self.probs = self.counts / X.shape[0]
        
        # Stats for class
        mean  = []
        cov   = []
        inv   = []
        det   = []
        
        #
        for c in self.classes:
            #
            subset = X[y.reshape(y.shape[0]) == c]
            
            #
            mean.append(subset.mean(axis=0))
            
            #
            cov_ = np.cov(subset.T)
            
            cov.append(cov_)                 # n x n
            inv.append(np.linalg.inv(cov_))  # n x n
            det.append(np.linalg.det(cov_))  # scalar
            
        # Converting arrays
        self.mean = np.array(mean)
        self.cov = np.array(cov)
        self.inv = np.array(inv)
        self.det = np.array(det)
        
    #
    def predict(self, X):
        y_hats = []
        part = []
        
        # For each class
        for i, c in enumerate(self.classes):
            part.append((np.sqrt((2 * np.pi)**X.shape[1] * self.det[i]))**-1)
            
        for x in X:
            preds = []
            
            for i, c in enumerate(self.classes):
                preds.append(part[i] * np.exp((x - clf.mean[i]).T.dot(clf.inv[i]).dot(x - clf.mean[i])*-0.5))
                
            y_hats.append(np.argmax(preds))
            
        return y_hats


### Naive Bayes Gaussiano

In [8]:
class GaussianNaiveBayes(BaseEstimator, RegressorMixin):
    ''' Gaussian Naive Bayes '''

    def fit(self, X, y):
        # Classes and Counts
        self.classes, self.counts = np.unique(y, return_counts=True)
        
        # Distributions (Prior)
        self.probs = self.counts / X.shape[0]
        
        # Stats for class
        mean = []
        var = []
        
        #
        for c in self.classes:
            #
            subset = X[y.reshape(y.shape[0]) == c]
            
            #
            mean.append(subset.mean(axis=0))
            var.append(subset.var(axis=0))
         
        # Converting arrays
        self.mean = np.array(mean)
        self.var = np.array(var)
            
    #
    def predict(self, X):
        # Posterior for each class
        y_hats = []
        
        #
        for i, c in enumerate(self.classes):
            prior = np.log(self.probs[i])
            
            c1 = np.log(2 * np.pi * self.var[i]).sum()/2
            c2 = ((((X - self.mean[i])**2)/self.var[i])).sum(axis=1)/2
            
            y_hats.append(prior - c1 - c2)

        #
        return np.argmax(np.array(y_hats), axis=0)

---

## Questão 1

Considere o conjunto de dados disponível em `breastcancer.csv`, organizado em 31 colunas, sendo as 30 primeiras colunas os atributos e a última coluna a saída. Os 30 atributos coletados de exames médicos são usados no diagnóstico do câncer de mama, sendo `1` a classe positiva e `0` a classe negativa.

#### Naive Bayes Gaussiano

In [9]:
# Importação dos conjuntos de dados
bcw = np.genfromtxt('breastcancer.csv', delimiter=',')
vehicle = np.genfromtxt('vehicle.csv', delimiter=',')

print(f'BCW: {bcw.shape}')
print(f'Vehicle: {vehicle.shape}')

BCW: (569, 31)
Vehicle: (846, 19)


### Questão 1 a.

Considerando uma validação cruzada em 10 folds, avalie modelos de classificação binária nos dados em questão. Para tanto, use as abordagens abaixo:
 - Regressão logística (treinado com GD ou SGD);
 - Análise do discriminante Gaussiano ;
 - Naive Bayes Gaussiano ;
 
### Questão 1 b.

Para cada modelo criado, reporte valor médio e desvio padrão da acurácia global e da acurácia por classe
 
[Início ↑](#Sumário)

In [10]:
X_bcw = bcw[:, :-1]
y_bcw = bcw[:, -1:]

#### Regressão Logistica

In [11]:
clf = BinaryLogisticRegressionGD(epocs=300)
_ = cross_val(clf, X_bcw, y_bcw, stats=True, progress=True)

---------------------------------------------------
 BinaryLogisticRegressionGD()
--------------------------------------------------
 Global Accuracy
  - Mean:               0.8120614035087719
  - Standard deviation: 0.07087134388319727
 Accuracy by class
  - Mean:               [0.75147913 0.90409341]
  - Standard deviation: [0.10383044 0.06414726]


#### Análise do discriminante Gaussiano

In [12]:
clf = GaussianDiscriminantAnalysis()
_ = cross_val(clf, X_bcw, y_bcw, stats=True, progress=True)

---------------------------------------------------
 GaussianDiscriminantAnalysis()
--------------------------------------------------
 Global Accuracy
  - Mean:               0.611748120300752
  - Standard deviation: 0.1150275986902135
 Accuracy by class
  - Mean:               [0.44744712 0.80878053]
  - Standard deviation: [0.23118794 0.13944683]


#### Naive Bayes Gaussiano

In [13]:
clf = GaussianNaiveBayes()
_ = cross_val(clf, X_bcw, y_bcw, stats=True, folds=10, progress=True)

---------------------------------------------------
 GaussianNaiveBayes()
--------------------------------------------------
 Global Accuracy
  - Mean:               0.7963972431077694
  - Standard deviation: 0.1007025420741757
 Accuracy by class
  - Mean:               [0.7112618  0.98365079]
  - Standard deviation: [0.14070952 0.02583013]


---

## Questão 2

Considere o conjunto de dados disponível em vehicle.csv , organizado em 19 colunas, sendo as 18 primeiras colunas os atributos e a última coluna a saída. Os 18 atributos caracterizam a silhueta de veículos, extraídos pelo método HIPS (Hierarchical Image Processing System). A tarefa consiste em classificar o veículo em 4 classes (bus, pel, saab, e van).

> Os dados já foram carregados

### Questão 2 a.

Considerando uma validação cruzada em 10 folds, avalie modelos de classi

cação multiclasse nos dados em questão. Para tanto, use as abordagens abaixo:

 - Regressão softmax (treinado com GD ou SGD);
 - Análise do discriminante Gaussiano;
 - Naive Bayes Gaussiano;

### Questão 2 b.

Para cada modelo criado, reporte valor médio e desvio padrão da acurácia global e da acurácia por classe

[Início ↑](#Sumário)

In [14]:
X_vehicle = vehicle[:, :-1]
y_vehicle = vehicle[:, -1:]

#### Regressão softmax (treinado com GD)

In [15]:
clf = SoftmaxRegressionGD(epocs=500)
_ = cross_val(clf, X_vehicle, y_vehicle, stats=True, progress=True)

---------------------------------------------------
 SoftmaxRegressionGD(epocs=500)
--------------------------------------------------
 Global Accuracy
  - Mean:               0.469313725490196
  - Standard deviation: 0.06377199798879717
 Accuracy by class
  - Mean:               [0.41929187 0.44579836 0.51681797 0.51623721]
  - Standard deviation: [0.08168527 0.09286956 0.11028727 0.13179864]


#### Análise do discriminante Gaussiano

In [16]:
clf = GaussianDiscriminantAnalysis()
_ = cross_val(clf, X_vehicle, y_vehicle, stats=True, progress=True)

---------------------------------------------------
 GaussianDiscriminantAnalysis()
--------------------------------------------------
 Global Accuracy
  - Mean:               0.5185294117647058
  - Standard deviation: 0.16947474405240176
 Accuracy by class
  - Mean:               [0.75306784 0.49360076 0.14528295 0.76269339]
  - Standard deviation: [0.19192543 0.40388856 0.2104879  0.26159325]


#### Naive Bayes Gaussiano

In [17]:
clf = GaussianNaiveBayes()
_ = cross_val(clf, X_vehicle, y_vehicle, stats=True)

--------------------------------------------------
 GaussianNaiveBayes()
--------------------------------------------------
 Global Accuracy
  - Mean:               0.42411764705882354
  - Standard deviation: 0.10234967545028588
 Accuracy by class
  - Mean:               [0.42163063 0.20453796 0.20654014 0.92622222]
  - Standard deviation: [0.10138334 0.16913912 0.18626918 0.09311741]
